In [3]:
import pandas as pd

In [31]:
def pega_data(linha):
    return linha[:15]

In [48]:
def pega_nome(linha):
    nome = ''
    for i in linha[18:]:
        if i == ':':
            return nome
        nome += i

In [64]:
def pega_texto(linha):
    linha2 = linha[18:]
    if ':' in linha2:
        return linha2[linha2.index(':') + 1:]
    else:
        return None

In [67]:
def cria_dic(linha):
    dic = {}
    dic['data'] = pega_data(linha)
    dic['nome'] = pega_nome(linha)
    dic['texto']= pega_texto(linha)
    
    return dic
    

In [72]:
arquivo = open('Conversa do WhatsApp com Conectivo.txt', 'r')

In [76]:
def cria_lista_dics(arquivo):
    lista = []
    for linha in arquivo.readlines():
        lista.append(cria_dic(linha))
        
    return lista

In [77]:
lista_dics = cria_lista_dics(arquivo)

In [79]:
df = pd.DataFrame(lista_dics)

In [81]:
df.head(10)

,data,nome,texto
0,"06/07/17, 16:38",None,None
1,"06/07/17, 16:38",None,None
2,"06/07/17, 16:38",None,None
3,"06/07/17, 16:38",Robson,kkk\n
4,"06/07/17, 16:38",Robson,xablau..\n
5,"06/07/17, 16:40",None,None
6,"06/07/17, 16:40",Camile,<Mídia omitida>\n
7,"06/07/17, 16:40",Vinicius Cognitivo. ai,camile\n
8,"06/07/17, 16:41",Vinicius Cognitivo. ai,o seu audio n fala nada\n
9,"06/07/17, 16:41",Vinicius Cognitivo. ai,só sai ruido\n
